<a href="https://colab.research.google.com/github/joao-albc/Bootcamp-Alura-Data-Science-Aplicada-2-Producao-Hospitalar/blob/main/Projetos/Projeto_01/Projeto_Mod_01_Alura_Bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução

## Projeto Bootcamp ALURA Módulo 1:

Explicar o que é a Alura e o Bootcamp

##Escopo proposto

**Módulo 01: Python e pandas para análise de dados reais**

*Neste módulo vamos aprender como utilizar **Python**, **Pandas** e **Matplotlib** para explorar os dados financeiros do SUS, mais precisamente os gastos por Unidade Federativa ao longo dos anos. Os desafios se iniciam na aquisição dos dados em uma plataforma do governo e seus possíveis problemas, passando pelo leitura destes dados com o Pandas até a criação de hipóteses na área da saúde, que sempre devem ser cuidadosas, simulando os desafios diários de uma pessoa cientista de dados.*

Alura (*https://bootcamps.alura.com.br/bootcamp-data-science-aplicada-2*)

## Objetivo:

Na primeira parte do curso, foram abordadas ( pontuar as aula). O objetivo deste projeto é colocar em prática o que foi aprendido no modulo 1.

## Dados:

Foi dada a opção de usar os dados de produção hospitalar, fosse número de internação, óbito, dias de permanência, taxa de mortalidade ou outros. Os dados ficam disponíveis no **portal da saúde do SUS** *(http://www2.datasus.gov.br/DATASUS/index.php?area=0202&id=11633&VObj=http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/qi)*

Fiz a escolha de selecionar os dados por número de óbitos. Estes, podem ser baixados no formato **.CSV**

#Exploração dos dados

#Importação de bibliotecas

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format = "{:.2f}".format

##Preparação dos dados

In [ ]:
obitos = pd.read_csv('A221901189_28_143_208.csv', encoding="ISO-8859-1", sep=";", skiprows = 3, skipfooter=12)
# O skiprows é usado para pular o cabeçalho

obitos.set_index('Unidade da Federação', inplace = True)
obitos.drop('Total', axis = 1, inplace = True)
obitos

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


FileNotFoundError: ignored

In [ ]:
investimento = pd.DataFrame(pd.read_csv('A171352189_28_143_208.csv',skiprows = 3,skipfooter= 12, encoding="ISO-8859-1",thousands=".", decimal=",",sep=";"))
investimento.set_index('Unidade da Federação', inplace = True)
investimento.drop('Total', axis = 1, inplace = True)
investimento

Substituição de valores em relação ao **12 Acre** no em **2009/Set**

A ideia é que os values sejam substituídos pelo valor média



Óbitos

In [ ]:
#variável para receber somente os valores do Estado (index)
n = obitos.loc['12 Acre'].values
#insere o valor com a média da array sem contar o valor ausente
obitos[obitos.columns[20]][1] = np.mean(np.delete(n,20))
# obitos[obitos.columns[20]]

In [ ]:
#variável para receber somente os valores do Estado (index)
n = investimento.loc['12 Acre'].values
#insere o valor com a média da array sem contar o valor ausente
investimento[investimento.columns[20]][1] = np.mean(np.delete(n,20))
# investimento[investimento.columns[20]]

##Inserir mês e ano

In [ ]:
def insert_month_year(df):
  meses = []
  for i in df.T.index:
    meses.append(re.search('\w*[a-zA-Z]',str(i)).group())

  ano = []
  for i in df.T.index:
    ano.append(int(re.search('\d*',str(i)).group()))

  df = df.T

  df['Mes'] = meses

  df['Ano'] = ano

  return df

In [ ]:

"""obitos = insert_month_year(obitos)
obitos"""


In [ ]:

"""investimento = insert_month_year(investimento)
investimento"""


##Inserir Região

In [ ]:
def insert_regiao(df):

  estados = df.index[:]
  regiao = []
  c = 0

  for i in estados:
    num = estados[c][0:2]
    if int(num) >= 11 and int(num) <= 17:
      regiao.append('Norte')
    elif int(num) >= 21 and int(num) <= 29:
      regiao.append('Nordeste')
    elif int(num) >= 31 and int(num) <= 35:
      regiao.append('Sudeste')
    elif int(num) >= 41 and int(num) <= 43:
      regiao.append('Sul')
    elif int(num) >= 50 and int(num) <= 53:
      regiao.append('Centro-Oeste')
    c+=1

  df['Região'] = regiao

  return df

In [ ]:
investimento = insert_regiao(investimento)
obitos = insert_regiao(obitos)

##**Análises por região**

**Explorar dados por região**

Podemos observar que existe um equilíbrio entre a média de investimento total e o número de óbitos por cada região do país. Na relação entre óbitos e investimentos, a região com maior sobra foi a Sul, com + 2.2% investidos. Por outro lado, o Sudeste registrou uma diferença de -4%.

In [ ]:
tabela_regiao = pd.DataFrame(obitos.groupby(['Região']).sum().T.sum().sort_values(), columns=['Óbitos.'])
tabela_regiao['% obt.'] = (obitos.groupby(['Região']).sum().T.sum().sort_values() / obitos[obitos.columns[0:-1]].sum().sum().round(4)*100)
tabela_regiao['Investimento'] = investimento.groupby(['Região']).sum().T.sum().sort_values()
tabela_regiao['% inv.'] = (investimento.groupby(['Região']).sum().T.sum().sort_values() / investimento.groupby(['Região']).sum().sum().sum())*100
tabela_regiao['% dif.'] = tabela_regiao['% inv.'] - tabela_regiao['% obt.']
tabela_regiao

In [ ]:
# obitos.groupby(['Região']).sum().T.sum().sort_values()


**Gráficos**

In [ ]:
fig, (ax,ax2) = plt.subplots(2, figsize=(10,6))
ax.set_title('Óbitos', loc='right', y=-0.1, pad=65.0, rotation = 90.0)
ax.bar(obitos.groupby(['Região']).sum().T.sum().sort_values().keys(),\
       obitos.groupby(['Região']).sum().T.sum().sort_values())

col = (obitos.groupby(['Região']).sum().T.sum().sort_values().keys())
val = (obitos.groupby(['Região']).sum().T.sum().sort_values().values)

#É usado ZIP para fazer a tupla com os dados de coluna e seu valor de maneira agregada
result = zip(col, val)
result = (tuple(result))
result

c = 0
for i in result:
  #Para inserir o rótulo de dados de cada uma das barras
  ax.annotate(str((val[c]/10000).round(2))+' k',xy=(c-0.2, val[c]*1.05))
  c+=1

ax2.set_title('Investimento em R$', loc='right', y=-0.1, pad=32.0, rotation = 90.0)
ax2.bar(investimento.groupby(['Região']).sum().T.sum().sort_values().keys(),\
       investimento.groupby(['Região']).sum().T.sum().sort_values())

col = (investimento.groupby(['Região']).sum().T.sum().sort_values().keys())
val = (investimento.groupby(['Região']).sum().T.sum().sort_values().values)

result = zip(col, val)
result = (tuple(result))
result

c = 0
for i in result:
  #Para inserir o rótulo de dados de cada uma das barras
  ax2.annotate(str((val[c]/1000000000).round(2))+' bi',xy=(c-0.2, val[c]*1.05))
  c+=1

plt.show()

Anos

Comparação por estado

In [ ]:
indice_cidade = dict(enumerate(tuple(investimento.index)))

In [ ]:
indice_cidade

In [ ]:
def grafico_combinado(cod_cidade):
  indice_cidade[cod_cidade]
  fig, ax = plt.subplots(figsize=(16,5), sharex=True)

  obts = obitos.T[:-1][indice_cidade[cod_cidade]].values
  inv = investimento.T[:-1][indice_cidade[cod_cidade]].values
  ax.plot(np.array(inv, dtype='float') / np.sum(inv, dtype='float') *100, label='Investimento')
  ax.plot(np.array(obts, dtype='float') / np.sum(obts, dtype='float') *100, label='Óbitos')
  
  ax.set_title(indice_cidade[cod_cidade])
  ax.legend()
  plt.show()

In [ ]:
n = list(indice_cidade.keys())[:7]

fig, ax = plt.subplots(1,int(len(n)), figsize=(20,5), sharey=True)
cod_cidade = 0
for i in n:
  obts = obitos.T[:-1][indice_cidade[cod_cidade]].values
  inv = investimento.T[:-1][indice_cidade[cod_cidade]].values
  ax[cod_cidade].plot(np.array(inv, dtype='float') / np.sum(inv, dtype='float') *100, label='Investimento')
  ax[cod_cidade].plot(np.array(obts, dtype='float') / np.sum(obts, dtype='float') *100, label='Óbitos')

  ax[cod_cidade].set_title(indice_cidade[cod_cidade])
  ax[cod_cidade].legend()
  cod_cidade+=1
plt.show()

In [ ]:
len(list(indice_cidade.keys())) / 9

In [ ]:
pd.read_csv('https://github.com/joao-albc/Bootcamp-Alura-Data-Science-Aplicada-2-Producao-Hospitalar/blob/main/Projetos/Projeto_01/Dados/A171352189_28_143_208.csv')

ParserError: ignored